<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="1000" alt="cognitiveclass.ai logo"  />
</center>

# Assignment: Exploratory Data Analysis with SQL

Estimated time needed: 60 minutes

# Introduction

Using this R notebook you will perform exploratory data analysis tasks using SQL queries with the RODBC R package. You will not be graded here, but you will be asked to present some of these results in your final assignment, where you will be graded on the accuracy of your results as well as the content of your SQL queries.


### Establish your Db2 connection

Load the 'RODBC' library, and use the 'odbcConnect( )' function as you did in the previous lab to establish the connection to your Db2 assets.\
Provided you successfully loaded your data into the tables in that lab, you are now ready to start running SQL queries using the RODBC library as you did in Course 3.


In [1]:
# provide your solution here
library(RODBC)

dsn_driver <- "{IBM DB2 ODBC Driver}"
dsn_database <- "bludb"            
dsn_hostname <- "2f3279a5-73d1-4859-88f0-a6c3e6b4b907.c3n41cmd0nqnrk39u98g.databases.appdomain.cloud" 
dsn_port <- "30756"                
dsn_protocol <- "TCPIP"            
dsn_uid <- "rnj71797"        
dsn_pwd <- "LLIyAugFgYYZcVYm" 
dsn_security <- "ssl"

conn_path <-paste("DRIVER=",dsn_driver,
               ";DATABASE=",dsn_database,
               ";HOSTNAME=",dsn_hostname,
               ";PORT=",dsn_port,
               ";PROTOCOL=",dsn_protocol,
               ";UID=",dsn_uid,
               ";PWD=",dsn_pwd,
               ";SECURITY=",dsn_security,
                sep= "")

conn <- odbcDriverConnect(conn_path,believeNRows=FALSE )


Warning message in odbcDriverConnect(conn_path, believeNRows = FALSE):
"[RODBC] ERROR: state IM002, code 0, message [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified"
Warning message in odbcDriverConnect(conn_path, believeNRows = FALSE):
"ODBC connection failed"


## Task 1 - Record Count

#### Determine how many records are in the seoul_bike_sharing dataset.


### Solution 1


In [2]:
# provide your solution here
count_records<-"select count (*) from SEOUL_BIKE_SHARING"
sqlQuery(conn,count_records)

,1
,<int>
1,8465


## Task 2 - Operational Hours

#### Determine how many hours had non-zero rented bike count.


### Solution 2


In [3]:
# provide your solution here
non_zero<-"select sum(RENTED_BIKE_COUNT) from SEOUL_BIKE_SHARING
            where HOUR > 0"
sqlQuery(conn,non_zero)

,1
,<int>
1,5974681


## Task 3 - Weather Outlook

#### Query the the weather forecast for Seoul over the next 3 hours.

Recall that the records in the CITIES_WEATHER_FORECAST dataset are 3 hours apart, so we just need the first record from the query.


### Solution 3


In [4]:
# provide your solution here
weather_outlook<-"select * from CITIES_WEATHER_FORECAST
                    limit 1"

sqlQuery(conn,weather_outlook)

,CITY,WEATHER,VISIBILITY,TEMP,TEMP_MIN,TEMP_MAX,PRESSURE,HUMIDITY,WIND_SPEED,WIND_DEG,SEASON,FORECAST_DATETIME
,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<int>,<fct>,<dttm>
1,Seoul,Clear,10000,12.32,10.91,12.32,1015,50,2.18,248,Spring,2021-04-16 12:00:00


## Task 4 - Seasons

#### Find which seasons are included in the seoul bike sharing dataset.


### Solution 4


In [45]:
# provide your solution here
seasons<-"select distinct SEASONS FROM SEOUL_BIKE_SHARING"

sqlQuery(conn,seasons)

,SEASONS
,<fct>
1,Autumn
2,Spring
3,Summer
4,Winter


## Task 5 - Date Range

#### Find the first and last dates in the Seoul Bike Sharing dataset.


### Solution 5


In [5]:
# provide your solution here
first_and_last<-"select MIN (DATE) from SEOUL_BIKE_SHARING where DATE like '%2017' 
                UNION 
                select MAX(DATE)as last from SEOUL_BIKE_SHARING where DATE like '%/11/2018' "

sqlQuery(conn,first_and_last)

,1
,<fct>
1,01/12/2017
2,30/11/2018


## Task 6 - Subquery - 'all-time high'

#### determine which date and hour had the most bike rentals.


### Solution 6


In [6]:
# provide your solution here
high_rental<-"select DATE from SEOUL_BIKE_SHARING 
                order by RENTED_BIKE_COUNT desc
                limit 1"

sqlQuery(conn,high_rental)

,DATE
,<fct>
1,19/06/2018


## Task 7 - Hourly popularity and temperature by season

#### Determine the average hourly temperature and the average number of bike rentals per hour over each season. List the top ten results by average bike count.


### Solution 7


In [24]:
# provide your solution here
popularity<-"select avg(TEMPERATURE)as hourly_temperature ,HOUR,avg(RENTED_BIKE_COUNT)as average_bike_rentals,SEASONS from SEOUL_BIKE_SHARING 
            group by HOUR,SEASONS
            order by average_bike_rentals desc
            limit 10"
    
sqlQuery(conn,popularity)

,HOURLY_TEMPERATURE,HOUR,AVERAGE_BIKE_RENTALS,SEASONS
,<dbl>,<int>,<int>,<fct>
1,29.38696,18,2135,Summer
2,16.03086,18,1983,Autumn
3,28.27283,19,1889,Summer
4,27.06630,20,1801,Summer
5,26.27826,21,1754,Summer
6,15.97222,18,1689,Spring
7,25.69891,22,1567,Summer
8,17.27778,17,1562,Autumn
9,30.07500,17,1526,Summer


## Task 8 - Rental Seasonality

#### Find the average hourly bike count during each season.

Also include the minimum, maximum, and standard deviation of the hourly bike count for each season.


### Solution 8


In [3]:
# provide your solution here
rental_season<-" select SEASONS,avg(RENTED_BIKE_COUNT)as HOURLY_BIKE,
                HOUR FROM SEOUL_BIKE_SHARING 
                group by HOUR,SEASONS 
                order by HOURLY_BIKE desc"

retrieve<-sqlQuery(conn,rental_season)

min(retrieve$HOURLY_BIKE)
max(retrieve$HOURLY_BIKE)
sd(retrieve$HOURLY_BIKE)

head(retrieve,10)

[1] 50

[1] 2135

[1] 512.0774

,SEASONS,HOURLY_BIKE,HOUR
,<fct>,<int>,<int>
1,Summer,2135,18
2,Autumn,1983,18
3,Summer,1889,19
4,Summer,1801,20
5,Summer,1754,21
6,Spring,1689,18
7,Summer,1567,22
8,Autumn,1562,17
9,Summer,1526,17


Let's explore a bit and see what might be the most significant contributing factors in terms of the provided data.


## Task 9 - Weather Seasonality

#### Consider the weather over each season. On average, what were the TEMPERATURE, HUMIDITY, WIND_SPEED, VISIBILITY, DEW_POINT_TEMPERATURE, SOLAR_RADIATION, RAINFALL, and SNOWFALL per season?

Include the average bike count as well , and rank the results by average bike count so you can see if it is correlated with the weather at all.


### Solution 9


In [4]:
query <- "select avg(TEMPERATURE)as temp ,avg(HUMIDITY)as humid,
            avg(WIND_SPEED)as wind,avg(VISIBILITY)as visibility,
            avg(DEW_POINT_TEMPERATURE)as dew_point,avg(SOLAR_RADIATION)as radiation,
            avg(RAINFALL)as rainfall ,avg(SNOWFALL)as snowfall,
            avg(RENTED_BIKE_COUNT)as rented_bike,seasons from SEOUL_BIKE_SHARING
            group by seasons order by rented_bike desc"
            

sqlQuery(conn,query)
# provide your solution here

,TEMP,HUMID,WIND,VISIBILITY,DEW_POINT,RADIATION,RAINFALL,SNOWFALL,RENTED_BIKE,SEASONS
,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<fct>
1,26.587274,64,1.609420,1501,18.750136,0.7612545,0.25348732,0.00000000,1034,Summer
2,13.821167,59,1.492101,1558,5.150594,0.5227827,0.11765617,0.06350026,924,Autumn
3,13.021389,58,1.857778,1240,4.091389,0.6803009,0.18694444,0.00000000,746,Spring
4,-2.540463,49,1.922685,1445,-12.416667,0.2981806,0.03282407,0.24750000,225,Winter


## Task 10 - Total Bike Count and City Info for Seoul

#### Use an implicit join across the WORLD_CITIES and the BIKE_SHARING_SYSTEMS tables to determine the total number of bikes avaialble in Seoul, plus the following city information about Seoul: CITY, COUNTRY, LAT, LON, POPULATION, in a single view.

Notice that in this case, the CITY column will work for the WORLD_CITIES table, but in general you would have to use the CITY_ASCII column.


### Solution 10


In [68]:
# provide your solution here
query<-"select b.country,b.city,b.bicycles,
        w.lat,w.lng,w.population
        from BIKE_SHARING_SYSTEMS b 
        inner JOIN WORLD_CITIES w
        ON b.city = w.city
        where b.city='Seoul'"

sqlQuery(conn,query)

,COUNTRY,CITY,BICYCLES,LAT,LNG,POPULATION
,<fct>,<fct>,<int>,<dbl>,<dbl>,<int>
1,South Korea,Seoul,20000,37.58,127,21794000


In [28]:
query<-" select BICYCLES from BIKE_SHARING_SYSTEMS where BICYCLES > '14500' "

sqlQuery(conn,query)

,BICYCLES
,<int>
1,16000
2,20000
3,15000
4,19165
5,22940
6,20000
7,20000
8,18000
9,20000


## Task 11 - Find all city names and coordinates with comparable bike scale to Seoul's bike sharing system

#### Find all cities with total bike counts between 15000 and 20000. Return the city and country names, plus the coordinates (LAT, LNG), population, and number of bicycles for each city.

Later we will ask you to visualize these similar cities on leaflet, with some weather data.


### Solution 11


In [29]:
# provide your solution here
query<- "select b.city,b.bicycles,
        w.country,w.lat,w.lng,w.population
        from BIKE_SHARING_SYSTEMS b 
        inner join WORLD_CITIES w
        on b.city= w.city
        where b.bicycles BETWEEN '15000' AND '20000' "

sqlQuery(conn,query)

,CITY,BICYCLES,COUNTRY,LAT,LNG,POPULATION
,<fct>,<int>,<fct>,<dbl>,<dbl>,<int>
1,Shanghai,19165,China,31.16,121.46,22120000
2,Seoul,20000,"Korea, South",37.58,127.00,21794000
3,Beijing,16000,China,39.90,116.39,19433000
4,Weifang,20000,China,36.71,119.10,9373000
5,Ningbo,15000,China,29.87,121.54,7639000
6,Zhuzhou,20000,China,27.84,113.14,3855609


In [24]:
close(conn)

## Author(s)

<h4> Jeff Grossman </h4>

## Contributor(s)

<h4> Yan Luo, Rav Ahuja </h4>

## Change log

| Date       | Version | Changed by    | Change Description                  |
| ---------- | ------- | ------------- | ----------------------------------- |
| 2021-05-04 | 0.2     | Jeff Grossman | Finalize                            |
| 2021-04-22 | 0.2     | Jeff Grossman | Update as per peer review           |
| 2021-04-10 | 0.2     | Jeff Grossman | Add data sources and implicit joins |
| 2021-04-05 | 0.1     | Jeff Grossman | Start content creation              |

## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
